In [134]:
# get forecasting data

import requests
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
import datetime
import json
import pandas as pd

today = datetime.datetime.now()
day_ago = today-datetime.timedelta(days=1)

base_time = '2000'
base_date = str(day_ago.year) + str(day_ago.month) + str(day_ago.day)

serviceKey = "52s6pZxbOiLmg7H76y%2BN1Rs2tXn4ru798V6iWXFOFWzwKDzqkzuVuxdhol%2FuBCEYc9IOUIKfeJBn2XcEzKXZbg%3D%3D"
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst'

queryParams = '?ServiceKey=' + serviceKey + '&pageNo=1&numOfRows=50000&dataType=JSON&base_date=' + base_date + '&base_time='+base_time + '&nx=62&ny=126'

response = requests.get(url+queryParams)

html = response.text
data = json.loads(html)
res = pd.DataFrame(data['response']['body']['items']['item'])

res['date'] = res['fcstDate'] + res['fcstTime']
res['date'] = pd.to_datetime(res['date'])
res = res.set_index('date')

# REH : 습도
# T3H : 기온
# WSD : 풍속

# 1. 습도
humidity = res[(res['category']=='REH')][['fcstValue']].astype(float)
humidity = humidity.resample('1H').first()
humidity = humidity.interpolate()

# 2. 기온
temperature = res[(res['category']=='T3H')][['fcstValue']].astype(float)
temperature = temperature.resample('1H').first()
temperature = temperature.interpolate()

# 3. 풍속
windSpeed = res[(res['category']=='WSD')][['fcstValue']].astype(float)
windSpeed = windSpeed.resample('1H').first()
windSpeed = windSpeed.interpolate()

# crate dataframe

newData = pd.DataFrame()
newData['date'] = humidity.index
newData = newData.set_index('date')
newData['temperature'] = temperature['fcstValue']
newData['humidity'] = humidity['fcstValue']
newData['windSpeed'] = windSpeed['fcstValue']
newData['weekday'] = newData.index.weekday
newData

# get holiday and join isHoliday in consumpData

serviceKey = "52s6pZxbOiLmg7H76y%2BN1Rs2tXn4ru798V6iWXFOFWzwKDzqkzuVuxdhol%2FuBCEYc9IOUIKfeJBn2XcEzKXZbg%3D%3D"
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'

startDate = newData.index[0]
endDate = newData.index[len(newData)-1]

start = datetime.date(startDate.year, startDate.month,1);
end = datetime.date(endDate.year, endDate.month,1);

# get holiday list
# 0:월, 1:화, 2:수, 3:목, 4:금, 5:토, 6:일

holidayList = [];
while(start<=end) :

    queryParams = '?ServiceKey=' + serviceKey + '&pageNo=1&numOfRows=100&solYear='+str(start.year)+'&solMonth='+str(start.month).zfill(2)
    
    response = requests.get(url+queryParams)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    holiday = soup.find_all('locdate')
    
    for i in range(0,len(holiday)) :
        
        holidayStr = holiday[i].text;
        year = holidayStr[0:4]
        month = holidayStr[4:6]
        day = holidayStr[6:8]
        
        holidayList.append(year+'-'+month+'-'+day);
        
    start = start + relativedelta(months=1)

newData['isHoliday'] = 0
newData.loc[newData['weekday']==5,'isHoliday'] = 1
newData.loc[newData['weekday']==6,'isHoliday'] = 1

for i in range(0,len(holidayList)) :
    newData[(newData.index)==i] = int(1)
    
newData = newData.astype({'isHoliday' : 'int'})
newData

KeyError: 'body'

In [17]:
day = datetime.datetime.now()
day_ago = day-datetime.timedelta(days=1)
day_ago

datetime.datetime(2020, 11, 28, 14, 46, 39, 436716)

In [19]:
base_time = '2000'
base_date = str(day_ago.year) + str(day_ago.month) + str(day_ago.day)
print(base_date+base_time)

202011282000


In [69]:
now = datetime.datetime.strptime('202011281200',"%Y%m%d%H%M%S")

In [70]:
now

datetime.datetime(2020, 11, 28, 12, 0)

In [73]:
res

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20201128,2000,POP,20201129,0000,20,62,126
1,20201128,2000,PTY,20201129,0000,0,62,126
2,20201128,2000,R06,20201129,0000,0,62,126
3,20201128,2000,REH,20201129,0000,55,62,126
4,20201128,2000,S06,20201129,0000,0,62,126
5,20201128,2000,SKY,20201129,0000,3,62,126
6,20201128,2000,T3H,20201129,0000,-1,62,126
7,20201128,2000,UUU,20201129,0000,0.9,62,126
8,20201128,2000,VEC,20201129,0000,312,62,126
9,20201128,2000,VVV,20201129,0000,-0.8,62,126


In [123]:
class Test :
    
    def __init__(self) :
        
    def testPrint() :
        print('kk')

In [132]:
test = Test

In [133]:
test.testPrint()

kk
